In [1]:
from utilities import *
from core.initial import Insertions
from core.solver import CGCPSolver
from docplex.cp.model import *
from docplex.mp.model import Model
import pandas as pd

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()
initial = Insertions(model)
initial.solve()
s = CGCPSolver(initial.sol)

In [ ]:
rmp = Model(name='Restricted_Master_Problem')

rmp.trips = s.trips
rmp.duties = s.duties

rmp.trip_duty_arr = rmp.binary_var_matrix(s.trips, s.duties)

In [ ]:
rmp.duty_arr = rmp.binary_var_list(rmp.duties, name='duty')

In [ ]:
for trip in rmp.trips:
    duty_ct = rmp.sum(rmp.trip_duty_arr[(trip, duty)] * rmp.duty_arr[duty.ID] for duty in rmp.duties) == 1
    rmp.add_constraint(duty_ct)

In [ ]:
rmp.minimize(rmp.sum(rmp.duty_arr[duty.ID] for duty in rmp.duties))

In [ ]:
rmp.solve()

In [3]:
sub = CpoModel(name="Pricing_Subproblem")

In [4]:
ntrips = len(model.trips)
nduties = len(model.trips)

In [5]:
trips = [interval_var(start=trip.start_time,
                      end=trip.end_time,
                      length=trip.duration,
                      size=trip.duration,
                      name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [6]:
duties = [interval_var(length=model.constraints.shift_span,
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}") for i in range(nduties)]

perform_duty = binary_var_list(nduties, name='Duty')


In [7]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [8]:
cdt = integer_var_list(nduties, min=0, max=model.constraints.continuous_driving, name="CDT")
tdt = integer_var_list(nduties, min=0, max=model.constraints.continuous_driving, name="TDT")

In [ ]:
for i in range(ntrips-1):
    sub.add(end_before_start(trips[i], trips[i + 1]))

In [ ]:
sub

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [ ]:
end_of(trips[0]).equals(trips[1])

In [ ]:
INTERVAL_MAX